In [ ]:
import numpy as np
import torch

In [ ]:
a = torch.ones(5)
b = np.array(a)
c = a.numpy()
print(type(b))
print(type(a))

<class 'numpy.ndarray'>
<class 'torch.Tensor'>


In [ ]:
a = np.ones(5)
b = torch.tensor(a)
print(b)
b = torch.from_numpy(a)
print(b)

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [ ]:
if torch.cuda.is_available:
  device = torch.device('cuda')
  a = torch.ones(6, device = device)
  b = torch.ones(6)
  b = b.to(device)
  c = a+b
  c = c.to('cpu')
  #c = c.cpu()
  c = c.numpy()
  print(c)
  print(type(c))

[2. 2. 2. 2. 2. 2.]
<class 'numpy.ndarray'>


In [ ]:
c = torch.ones(5,requires_grad = True)

In [ ]:
import math
x = torch.randn(3, requires_grad = True)
print(x)
y = x+2
print(y)
z = torch.sin(y**2)
z = z.mean()
z.backward() #dz/dx

tensor([0.6941, 0.3010, 0.3399], requires_grad=True)
tensor([2.6941, 2.3010, 2.3399], grad_fn=<AddBackward0>)


In [ ]:
weights = torch.ones(4, requires_grad = True)
#weight = torch.sum(weights)
#weight=weights
for epoch in range(2):
  model_output = (weights*3).sum()
  model_output.backward()
  print(weights.grad)
  weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


In [ ]:
x = torch.tensor(2.0)
y = torch.tensor(10.0)
w = torch.tensor(4.0,requires_grad = True)
y_hat = w*x
loss = (y_hat-y)**2

w.grad # dloss/dw

None


tensor(-8.)

In [ ]:
x = torch.tensor([2.0, 3.0])
y = torch.tensor(10.0)

w = torch.tensor([3.0, 1.0], requires_grad=True)

#y_hat = w[0] * x[0] + w[1] * x[1]

y_hat = torch.dot(w,x)
loss = (y_hat - y) ** 2


loss.backward()

print(f"Initialized weights: {w}")
print(f"Gradients: {w.grad}")


Initialized weights: tensor([3., 1.], requires_grad=True)
Gradients: tensor([-4., -6.])


In [ ]:
print(np.dot(np.array([1,2]),np.array([2,3])))
print(np.array([1,2])*np.array([2,3]))

8
[2 6]


<h1 style="font-size:1.125rem;">Linear Regression from Scratch</h1>


In [ ]:
# Set X, Y and W(weight)
# Suppose y = 2*x
X = np.array([1,3,4,6],dtype=np.float32)
Y = np.array([2,6,8,12],dtype=np.float32)
W = 1.0
#forward
def forward(W,X):
  return np.dot(W,X)
#forward(W,X)
#loss
def MSE(Y_hat,Y):
  return np.array((Y_hat-Y)**2).mean()
#gradient
def gradient(X,Y_hat,Y):
  return 2*np.dot(X,(Y_hat-Y))
epochs = 20
lr = 0.01
for epoch in range(epochs):
  y_hat = forward(W,X)
  loss = MSE(y_hat, Y)
  grad = gradient(X,y_hat,Y)
  #update using GD
  W -= lr*grad
  print(f'Epoch {epoch+1} --- Prediction: {forward(W,5)}, W = {W:.4f}, loss = {loss:.4f}')


Epoch 1 --- Prediction: 11.200000000000001, W = 2.2400, loss = 15.5000
Epoch 2 --- Prediction: 9.712, W = 1.9424, loss = 0.8928
Epoch 3 --- Prediction: 10.06912, W = 2.0138, loss = 0.0514
Epoch 4 --- Prediction: 9.983411199999999, W = 1.9967, loss = 0.0030
Epoch 5 --- Prediction: 10.003981312, W = 2.0008, loss = 0.0002
Epoch 6 --- Prediction: 9.999044485119999, W = 1.9998, loss = 0.0000
Epoch 7 --- Prediction: 10.0002293235712, W = 2.0000, loss = 0.0000
Epoch 8 --- Prediction: 9.999944962342912, W = 2.0000, loss = 0.0000
Epoch 9 --- Prediction: 10.0000132090377, W = 2.0000, loss = 0.0000
Epoch 10 --- Prediction: 9.999996829830952, W = 2.0000, loss = 0.0000
Epoch 11 --- Prediction: 10.000000760840571, W = 2.0000, loss = 0.0000
Epoch 12 --- Prediction: 9.999999817398264, W = 2.0000, loss = 0.0000
Epoch 13 --- Prediction: 10.000000043824414, W = 2.0000, loss = 0.0000
Epoch 14 --- Prediction: 9.99999998948214, W = 2.0000, loss = 0.0000
Epoch 15 --- Prediction: 10.000000002524285, W = 2.000

<h1 style="font-size:1.125rem;">Linear Regression with Pytorch</h1>


In [ ]:
# Set X, Y and W(weight)
# Suppose y = 2*x
import torch.nn as nn
X = torch.tensor([[1],[3],[4],[6]],dtype=torch.float32)
Y = torch.tensor([[2],[6],[8],[12]],dtype=torch.float32)
X_test = torch.tensor([[5]],dtype=torch.float32)
#W = torch.tensor([5],requires_grad=True)
epochs = 20
lr = 0.01
sample,feature = X.shape

model = nn.Linear(feature,feature)
print(f'Predictions before training: {model(X_test).item():.3f}')

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=lr)
for epoch in range(epochs):
  #forward
  y_pred = model(X)
  #loss
  l = loss(Y,y_pred)
  #gradient
  l.backward() #dl/dw
  optimizer.step()
  optimizer.zero_grad()
  w,b=model.parameters()
  #print(f'Epoch {epoch+1} --- Prediction: {y_pred.squeeze().detach().numpy()}, W = {w[0][0].item():.4f}, loss = {l.item():.4f}')
  print(f'Epoch {epoch+1} --- Prediction: {model(X_test).item():.4f}, W = {w[0][0].item():.4f}, loss = {l.item():.4f}')




Predictions before training: -4.059
Epoch 1 --- Prediction: 0.5067, W = 0.1067, loss = 123.0573
Epoch 2 --- Prediction: 3.5838, W = 0.6955, loss = 55.9168
Epoch 3 --- Prediction: 5.6577, W = 1.0925, loss = 25.4176
Epoch 4 --- Prediction: 7.0556, W = 1.3601, loss = 11.5628
Epoch 5 --- Prediction: 7.9979, W = 1.5406, loss = 5.2691
Epoch 6 --- Prediction: 8.6330, W = 1.6624, loss = 2.4100
Epoch 7 --- Prediction: 9.0611, W = 1.7446, loss = 1.1110
Epoch 8 --- Prediction: 9.3498, W = 1.8001, loss = 0.5209
Epoch 9 --- Prediction: 9.5444, W = 1.8376, loss = 0.2526
Epoch 10 --- Prediction: 9.6756, W = 1.8630, loss = 0.1307
Epoch 11 --- Prediction: 9.7641, W = 1.8802, loss = 0.0751
Epoch 12 --- Prediction: 9.8239, W = 1.8920, loss = 0.0498
Epoch 13 --- Prediction: 9.8642, W = 1.9000, loss = 0.0381
Epoch 14 --- Prediction: 9.8914, W = 1.9055, loss = 0.0327
Epoch 15 --- Prediction: 9.9099, W = 1.9093, loss = 0.0301
Epoch 16 --- Prediction: 9.9223, W = 1.9120, loss = 0.0289
Epoch 17 --- Prediction:

<h1 style="font-size:1.125rem;">Logistic Regression with Pytorch</h1>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
